In [8]:
import pandas as pd
import random
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import plotly.express as px
from sklearn.preprocessing import MultiLabelBinarizer

# 데이터 로딩

In [2]:
data_dir = "/data/ephemeral/level2-movierecommendation-recsys-06/data/train/"
movie = pd.read_csv(data_dir + "movie.csv")[["item", "genre", "year"]]

# 데이터 전처리

In [3]:
# 다중 장르를 이진 변수로 변환
mlb = MultiLabelBinarizer()
genre_matrix = mlb.fit_transform(movie['genre'])
genre_df = pd.DataFrame(genre_matrix, columns=mlb.classes_)

# 원래 데이터프레임과 장르 데이터프레임을 합치기
df_merged = pd.concat([movie, genre_df], axis=1)

# t-SNE를 적용하기 전에 데이터 정규화 (예: Min-Max 스케일링)
df_normalized = (df_merged.iloc[:, 3:] - df_merged.iloc[:, 3:].min()) / (df_merged.iloc[:, 3:].max() - df_merged.iloc[:, 3:].min())

# T-SNE

## 2차원

In [15]:
# t-SNE 모델 초기화 및 학습
tsne = TSNE(n_components=2, random_state=42)
tsne_result = tsne.fit_transform(df_normalized)

# 클러스터링을 위해 KMeans 모델 초기화 및 학습
kmeans = KMeans(n_clusters=20, random_state=42)
movie['cluster'] = kmeans.fit_predict(df_normalized)

# 결과를 새로운 데이터프레임에 추가
df_tsne = pd.DataFrame(tsne_result, columns=['t-SNE1', 't-SNE2'])
df_tsne[['item', 'year', 'cluster']] = movie[['item', 'year', 'cluster']]

# 명명된 색상 팔레트 사용
color_palette = px.colors.qualitative.Set1  # Set1은 명명된 색상 팔레트 중 하나입니다.

# 2D t-SNE를 생성하여 툴팁, 클러스터별 색상, 점 크기 설정
fig = px.scatter(df_tsne, x='t-SNE1', y='t-SNE2', color='cluster', 
                 hover_data=['item'], size_max=8, size='cluster',
                 color_discrete_sequence=color_palette)

fig.update_layout(title='2D t-SNE 결과')

fig.show()

## 3차원

In [13]:
# t-SNE 모델 초기화 및 학습 (3D로 수정)
tsne = TSNE(n_components=3, random_state=42)
tsne_result = tsne.fit_transform(df_normalized)

# 클러스터링을 위해 KMeans 모델 초기화 및 학습
kmeans = KMeans(n_clusters=20, random_state=42)
movie['cluster'] = kmeans.fit_predict(df_normalized)

# 결과를 새로운 데이터프레임에 추가
df_tsne = pd.DataFrame(tsne_result, columns=['t-SNE1', 't-SNE2', 't-SNE3'])
df_tsne[['item', 'year', 'cluster']] = movie[['item', 'year', 'cluster']]

# 각 클러스터에 대한 랜덤한 색상 생성
cluster_colors = {cluster: f'#{random.randint(0, 0xFFFFFF):06x}' for cluster in df_tsne['cluster'].unique()}

# 3D t-SNE를 생성하여 툴팁, 클러스터별 색상, 점 크기 설정
fig = px.scatter_3d(df_tsne, x='t-SNE1', y='t-SNE2', z='t-SNE3', color='cluster', color_discrete_map=cluster_colors, 
                    hover_data=['item'], size_max=8, size='cluster')
fig.update_layout(title='3D t-SNE 결과')

fig.show()